## Adjusting for merging new Scraping from Perigon with old pre-2016 articles

In [1]:
import os
import json
import pandas as pd

In [2]:
df_combined = pd.read_csv('df_combined_perigon')
df_combined_old = pd.read_csv('df_combined_old.csv')

In [4]:
print(len(df_combined))
print(len(df_combined_old))

12054
8338


In [5]:
df_combined_old_telegraph = df_combined_old[df_combined_old['outlet']=='telegraph']
df_combined_new_telegraph = df_combined[df_combined['outlet']=='telegraph']


In [6]:
df_combined_old_telegraph

,date,title,body,outlet,word_count,year,month,title_readability,body_readability
4191,2019-11-21,Boris Johnson can unite the country on immigra...,Democratic politics rewards politicians who c...,telegraph,718,2019.0,2019-11,63.36,44.48
4192,2023-01-15,Two thirds of tagged Albanian migrants have cu...,Two thirds of the Albanian Channel migrants w...,telegraph,561,2023.0,2023-01,74.19,45.59
4193,2017-07-09,Armed US immigration officers stationed at Bri...,British holidaymakers visiting the US could b...,telegraph,771,2017.0,2017-07,59.64,50.87
4194,2021-01-26,Heathrow crowding due to Covid test checks on ...,A row over health risks from Heathrow passpor...,telegraph,396,2021.0,2021-01,76.22,59.67
4195,2017-10-06,Polish migrant who threatened 'to kill an MP' ...,A Polish migrant who threatened Downing Stree...,telegraph,539,2017.0,2017-10,74.19,66.47
...,...,...,...,...,...,...,...,...,...
8333,2020-01-26,Priti Patel says UK firms 'too reliant' on che...,British companies have become far too reliant...,telegraph,716,2020.0,2020-01,49.15,48.37
8334,2023-07-08,Rishi Sunak is a Conservative Brexiteer doomed...,"Let me advance a suggestion that, in the curr...",telegraph,1277,2023.0,2023-07,64.71,62.68
8335,2024-01-14,Five migrants die in first known Channel cross...,Five migrants have died and a sixth in critic...,telegraph,525,2024.0,2024-01,84.68,58.62
8336,2018-12-06,Migrant rescue vessel Aquarius ends operations...,The Aquarius migrant rescue ship is to end it...,telegraph,370,2018.0,2018-12,29.52,36.22


In [7]:
df_combined_new_telegraph

,Unnamed: 0,date,title,body,outlet
6027,6027,2024-01-30 00:00:00,Tuesday evening news briefing: UK population c...,Good evening. Net migration is on course to d...,telegraph
6028,6028,2023-11-27 00:00:00,Monday evening news briefing: Growth outlook i...,Good evening. Andrew Bailey has said the outl...,telegraph
6029,6029,NaN,Muslim model Halima Aden: ‘Not even for $10 mi...,"Halima Aden, the Hijabi model who appeared on...",telegraph
6030,6030,NaN,A GP visit has become as random as booking an ...,Remember the days when going to the doctors m...,telegraph
6031,6031,2020-09-11 00:00:00,Tom Hollander interview: ‘The future of theatr...,Tom Hollander is the consummate showstealer. ...,telegraph
...,...,...,...,...,...
12049,12049,2022-11-08 00:00:00,Emmanuel Macron hails new era of 'ambitious co...,Emmanuel Macron agreed to boost joint efforts...,telegraph
12050,12050,2023-05-14 00:00:00,Suella Braverman challenges Sunak to cut ‘unsu...,Suella Braverman will demand that Rishi Sunak...,telegraph
12051,12051,2024-03-26 00:00:00,Churches risk undermining asylum system after ...,Churches risk undermining the integrity of th...,telegraph
12052,12052,2021-04-07 00:00:00,Exclusive: Satellite images reveal how shadowy...,Britain should confront Russian mercenary gro...,telegraph


In [8]:
# Adjust dates for new
df_combined_new_telegraph['date'] = pd.to_datetime(df_combined_new_telegraph['date'], errors='coerce')
df_combined_new_telegraph['date'] = df_combined_new_telegraph['date'].dt.strftime('%Y-%m-%d')

/tmp/ipykernel_3047427/935227890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined_new_telegraph['date'] = pd.to_datetime(df_combined_new_telegraph['date'], errors='coerce')
/tmp/ipykernel_3047427/935227890.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined_new_telegraph['date'] = df_combined_new_telegraph['date'].dt.strftime('%Y-%m-%d')


In [9]:
num_nan_dates = df_combined_new_telegraph['date'].isna().sum()
print(f"Number of rows with NaN in 'date' column: {num_nan_dates}")

Number of rows with NaN in 'date' column: 963


In [11]:
df_combined = pd.concat([df_combined_new_telegraph, df_combined_old_telegraph], ignore_index=True)
df_combined_final_telegraph = df_combined.drop_duplicates(subset='body', keep='first')
df_combined_final_telegraph.to_csv('df_Telegraph.csv', index=False)
df_combined_final_telegraph.shape

(8336, 10)

In [12]:
num_nan_dates = df_combined_final_telegraph['date'].isna().sum()
print(f"Number of rows with NaN in 'date' column: {num_nan_dates}")

Number of rows with NaN in 'date' column: 1134


In [13]:
import re
def clean_text(text):
    if not isinstance(text, str):
        return '' 
    cleaned_text = text.replace('\n', ' ')
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s.,?]', '', cleaned_text)
    return cleaned_text

In [14]:
df_Telegraph = df_combined_final_telegraph
df_Telegraph['body'] = df_Telegraph['body'].apply(clean_text)

/tmp/ipykernel_3047427/2680751308.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Telegraph['body'] = df_Telegraph['body'].apply(clean_text)


In [15]:
df_Telegraph = df_Telegraph.sample(frac=1, random_state=42)
df_Telegraph.to_csv('df_Telegraph_final', index=False)

In [16]:
file_path = "guardian_articles.csv"
df_guardian = pd.read_csv(file_path)

# Understanding time range of data
minimum_date = df_guardian['Date'].min()
maximum_date = df_guardian['Date'].max()

print("Minimum Date:", minimum_date)
print("Maximum Date:", maximum_date)
df_guardian.rename(columns={'Date': 'date', 'Title': 'title', 'Body': 'body'}, inplace=True)

df_guardian.dropna(subset=['body'], inplace=True)
df_guardian['body'] = df_guardian['body'].apply(clean_text)
# Randomly shuffle the rows of the DataFrame
df_guardian = df_guardian.sample(frac=1, random_state=42)
df_guardian_sampled = df_guardian.sample(n=8336, random_state=42)
print(len(df_guardian_sampled))

df_guardian_sampled['outlet'] = 'guardian'
df_Telegraph['outlet'] = 'telegraph'
df_guardian_sampled['date'] = pd.to_datetime(df_guardian_sampled['date'], errors='coerce')

Minimum Date: 2016-07-07T14:36:52Z
Maximum Date: 2024-04-27T14:00:36Z
8336


In [27]:
df_guardian_selected = df_guardian_sampled[['date', 'title', 'body', 'outlet']]
df_telegraph_selected = df_Telegraph[['date', 'title', 'body', 'outlet']]

In [28]:
df_telegraph_selected.to_csv('df_telegraph_perigon')
df_guardian_selected.to_csv('df_guardian_perigon')

In [29]:
print(len(df_telegraph_selected))
print(len(df_guardian_selected))

8336
8336


In [2]:
pip uninstall cupy cupy-cuda111


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install cupy-cuda111

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.1/101.1 MB 39.6 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 14.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [30]:
df_combined = pd.concat([df_guardian_selected, df_telegraph_selected], ignore_index=True)
print(len(df_combined))

df_combined.to_csv('df_combined_perigon_full')

16672
